In [3]:
!pip install python-docx



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_comments_and_highlighted_text(docx_file):
    doc = Document(docx_file)
    
    # Extract comments
    comments = []
    for comment in doc.comments:
        comments.append({
            'author': comment.author,
            'text': comment.text
        })
    
    # Extract highlighted text
    highlighted_text = []
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                highlighted_text.append(run.text)
    
    return comments, highlighted_text



In [6]:
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_highlighted_text(docx_file):
    doc = Document(docx_file)
    
    highlighted_text = []
    
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                highlighted_text.append(run.text)
    
    return highlighted_text

# Example usage
if __name__ == "__main__":
    docx_file = 'Document.docx'
    highlighted_text = extract_highlighted_text(docx_file)
    
    print("HIGHLIGHTED TEXT:")
    for text in highlighted_text:
        print(text)

HIGHLIGHTED TEXT:
 software 
. Wikipedia is the largest and most-read 
 in history,
 and is consistently ranked among the ten 
; as of May 2024, it was ranked fourth by 
,
 and sixth by 
.
 Founded by 
 and 
 on January 15, 2001, Wikipedia has been hosted since 2003 by the 
, an American 
 funded mainly by donations from readers.


In [9]:
from zipfile import ZipFile
from lxml import etree

def extract_comments(docx_file):
    comments = []

    # Open the docx file as a zip archive
    with ZipFile(docx_file, 'r') as zipf:
        # Check if 'word/comments.xml' exists in the zip file
        if 'word/comments.xml' in zipf.namelist():
            with zipf.open('word/comments.xml') as f:
                # Parse the XML using lxml
                tree = etree.parse(f)
                root = tree.getroot()

                # Find all comments in the document
                for comment in root.findall('.//w:comment', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                    comment_id = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', '')
                    author = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}author', '')
                    date = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}date', '')

                    # Extract text of the comment
                    comment_text = ""
                    for text_element in comment.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        if text_element.text:
                            comment_text += text_element.text

                    comments.append({
                        'id': comment_id,
                        'author': author,
                        'date': date,
                        'text': comment_text
                    })
        else:
            print("No comments.xml found in the document.")

    return comments

# Example usage
if __name__ == "__main__":
    docx_file = 'Document.docx'
    comments = extract_comments(docx_file)
    
    print("COMMENTS:")
    for comment in comments:
        print(f"Comment ID: {comment['id']}")
        print(f"Author: {comment['author']}")
        print(f"Date: {comment['date']}")
        print(f"Text: {comment['text']}")
        print()


COMMENTS:
Comment ID: 1
Author: Hrohaan Malhotra
Date: 2024-06-17T05:41:59Z
Text: This is comment number 2

Comment ID: 0
Author: Hrohaan Malhotra
Date: 2024-06-17T05:41:43Z
Text: This is a Comment



In [10]:
from zipfile import ZipFile
from lxml import etree
import pandas as pd
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_comments_and_highlighted_text(docx_file):
    comments = []
    highlighted_text = []

    # Extract comments using lxml
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/comments.xml' in zipf.namelist():
            with zipf.open('word/comments.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                for comment in root.findall('.//w:comment', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                    comment_id = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', '')
                    author = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}author', '')
                    date = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}date', '')

                    comment_text = ""
                    for text_element in comment.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        if text_element.text:
                            comment_text += text_element.text

                    comments.append({
                        'Type': 'Comment',
                        'ID': comment_id,
                        'Author': author,
                        'Date': date,
                        'Text': comment_text
                    })

        else:
            print("No comments.xml found in the document.")

    # Extract highlighted text using python-docx
    doc = Document(docx_file)
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                highlighted_text.append({
                    'Type': 'Highlighted Text',
                    'Text': run.text
                })

    # Create a DataFrame
    df_comments = pd.DataFrame(comments)
    df_highlighted_text = pd.DataFrame(highlighted_text)
    
    return df_comments, df_highlighted_text

# Example usage
if __name__ == "__main__":
    docx_file = 'Document.docx'
    df_comments, df_highlighted_text = extract_comments_and_highlighted_text(docx_file)
    
    print("COMMENTS:")
    print(df_comments)
    print()
    
    print("HIGHLIGHTED TEXT:")
    print(df_highlighted_text)


COMMENTS:
      Type ID            Author                  Date  \
0  Comment  1  Hrohaan Malhotra  2024-06-17T05:41:59Z   
1  Comment  0  Hrohaan Malhotra  2024-06-17T05:41:43Z   

                       Text  
0  This is comment number 2  
1         This is a Comment  

HIGHLIGHTED TEXT:
                Type                                               Text
0   Highlighted Text                                          software 
1   Highlighted Text          . Wikipedia is the largest and most-read 
2   Highlighted Text                                        in history,
3   Highlighted Text          and is consistently ranked among the ten 
4   Highlighted Text         ; as of May 2024, it was ranked fourth by 
5   Highlighted Text                                                  ,
6   Highlighted Text                                      and sixth by 
7   Highlighted Text                                                  .
8   Highlighted Text                                        F

In [11]:
from zipfile import ZipFile
from lxml import etree
import pandas as pd
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_comments_and_highlighted_text(docx_file):
    data = []

    # Extract comments using lxml
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/comments.xml' in zipf.namelist():
            with zipf.open('word/comments.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                for comment in root.findall('.//w:comment', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                    comment_id = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', '')
                    author = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}author', '')
                    date = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}date', '')

                    comment_text = ""
                    for text_element in comment.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        if text_element.text:
                            comment_text += text_element.text

                    data.append({
                        'Type': 'Comment',
                        'ID': comment_id,
                        'Author': author,
                        'Date': date,
                        'Text': comment_text
                    })

        else:
            print("No comments.xml found in the document.")

    # Extract highlighted text using python-docx
    doc = Document(docx_file)
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                data.append({
                    'Type': 'Highlighted Text',
                    'ID': None,
                    'Author': None,
                    'Date': None,
                    'Text': run.text
                })

    # Create a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Example usage
if __name__ == "__main__":
    docx_file = 'Document.docx'
    df = extract_comments_and_highlighted_text(docx_file)
    
    print("COMMENTS AND HIGHLIGHTED TEXT:")
    print(df)


COMMENTS AND HIGHLIGHTED TEXT:
                Type    ID            Author                  Date  \
0            Comment     1  Hrohaan Malhotra  2024-06-17T05:41:59Z   
1            Comment     0  Hrohaan Malhotra  2024-06-17T05:41:43Z   
2   Highlighted Text  None              None                  None   
3   Highlighted Text  None              None                  None   
4   Highlighted Text  None              None                  None   
5   Highlighted Text  None              None                  None   
6   Highlighted Text  None              None                  None   
7   Highlighted Text  None              None                  None   
8   Highlighted Text  None              None                  None   
9   Highlighted Text  None              None                  None   
10  Highlighted Text  None              None                  None   
11  Highlighted Text  None              None                  None   
12  Highlighted Text  None              None               

In [12]:
df.head(1)

,Type,ID,Author,Date,Text
0,Comment,1,Hrohaan Malhotra,2024-06-17T05:41:59Z,This is comment number 2


In [22]:
from zipfile import ZipFile
from lxml import etree
import pandas as pd
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_comments_and_highlighted_text(docx_file):
    data = []

    # Extract comments using lxml
    comments = {}
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/comments.xml' in zipf.namelist():
            with zipf.open('word/comments.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                for comment in root.findall('.//w:comment', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                    comment_id = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', '')
                    author = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}author', '')
                    date = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}date', '')

                    comment_text = ""
                    for text_element in comment.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        if text_element.text:
                            comment_text += text_element.text

                    comments[comment_id] = {
                        'Author': author,
                        'Date': date,
                        'Text': comment_text
                    }
        else:
            print("No comments.xml found in the document.")

    # Extract highlighted text and match it to comments
    doc = Document(docx_file)
    highlighted_texts = []
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                highlighted_texts.append(run.text)

    # Pair highlighted text with comments
    for i, (comment_id, comment_data) in enumerate(comments.items()):
        if i < len(highlighted_texts):
            data.append({
                'Highlighted_Text': highlighted_texts[i],
                'Comment Text': comment_data['Text'],
                'Comment Author': comment_data['Author'],
                'Comment Date': comment_data['Date']
            })
        else:
            data.append({
                'Highlighted_Text': 'N/A',
                'Comment Text': comment_data['Text'],
                'Comment Author': comment_data['Author'],
                'Comment Date': comment_data['Date']
            })

    # Create a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Example usage
if __name__ == "__main__":
    docx_file = 'Document2.docx'
    df = extract_comments_and_highlighted_text(docx_file)
    
    print("HIGHLIGHTED TEXT AND COMMENTS PAIRS:")
    print(df)


HIGHLIGHTED TEXT AND COMMENTS PAIRS:
                                    Highlighted_Text  \
0  That’s why this moment on the internet feels e...   
1  It’s easy to name a root cause — ads reward an...   
2  Now we have decades of proof that attention-gr...   
3  Everything Medium does is paid for by our memb...   

               Comment Text    Comment Author          Comment Date  
0         This is comment 4  Hrohaan Malhotra  2024-06-17T05:58:41Z  
1  This is comment number 3  Hrohaan Malhotra  2024-06-17T05:58:32Z  
2  This is Comment number 2  Hrohaan Malhotra  2024-06-17T05:58:20Z  
3  This is comment number 1  Hrohaan Malhotra  2024-06-17T05:58:09Z  


In [24]:
df.Highlighted_Text[1]

'It’s easy to name a root cause — ads reward any content that can grab your attention long enough to show you yet another ad, and the more the better.'

In [25]:
df.Highlighted_Text[0]

'That’s why this moment on the internet feels even more urgent than normal. Even before Google used AI to tell us to (really!), search results were already flooded with content that was written by or for machines, not humans.'

In [26]:
df.Highlighted_Text[2]

'Now we have decades of proof that attention-grabbing isn’t the same as good. Instead of the information superhighway that we were promised, ads gave us an internet where almost all incentives are to create cheap, high-volume, low-quality content designed to get as many eyeballs as possible.'

In [27]:
df.Highlighted_Text[3]

'Everything Medium does is paid for by our members, not advertising. We’re not trying to manipulate your attention to show you more ads. Instead, we care about a much harder challenge: How can we show you a story that you will be happy to have paid to read?'

In [28]:
from zipfile import ZipFile
from lxml import etree
import pandas as pd
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_comments_and_highlighted_text(docx_file):
    data = []

    # Extract comments using lxml
    comments = []
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/comments.xml' in zipf.namelist():
            with zipf.open('word/comments.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                for comment in root.findall('.//w:comment', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                    comment_id = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', '')
                    author = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}author', '')
                    date = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}date', '')

                    comment_text = ""
                    for text_element in comment.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        if text_element.text:
                            comment_text += text_element.text

                    comments.append({
                        'ID': comment_id,
                        'Author': author,
                        'Date': date,
                        'Text': comment_text
                    })
        else:
            print("No comments.xml found in the document.")

    # Extract highlighted text using python-docx
    highlighted_texts = []
    doc = Document(docx_file)
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                highlighted_texts.append(run.text)

    # Reverse the comments list
    comments.reverse()

    # Pair highlighted text with comments
    for i in range(max(len(highlighted_texts), len(comments))):
        if i < len(highlighted_texts) and i < len(comments):
            data.append({
                'Highlighted Text': highlighted_texts[i],
                'Comment Text': comments[i]['Text'],
                'Comment Author': comments[i]['Author'],
                'Comment Date': comments[i]['Date']
            })
        elif i < len(highlighted_texts):
            data.append({
                'Highlighted Text': highlighted_texts[i],
                'Comment Text': 'N/A',
                'Comment Author': 'N/A',
                'Comment Date': 'N/A'
            })
        else:
            data.append({
                'Highlighted Text': 'N/A',
                'Comment Text': comments[i]['Text'],
                'Comment Author': comments[i]['Author'],
                'Comment Date': comments[i]['Date']
            })

    # Create a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Example usage
if __name__ == "__main__":
    docx_file = 'Document2.docx'
    df = extract_comments_and_highlighted_text(docx_file)
    
    print("HIGHLIGHTED TEXT AND COMMENTS PAIRS:")
    print(df)


HIGHLIGHTED TEXT AND COMMENTS PAIRS:
                                    Highlighted Text  \
0  That’s why this moment on the internet feels e...   
1  It’s easy to name a root cause — ads reward an...   
2  Now we have decades of proof that attention-gr...   
3  Everything Medium does is paid for by our memb...   

               Comment Text    Comment Author          Comment Date  
0  This is comment number 1  Hrohaan Malhotra  2024-06-17T05:58:09Z  
1  This is Comment number 2  Hrohaan Malhotra  2024-06-17T05:58:20Z  
2  This is comment number 3  Hrohaan Malhotra  2024-06-17T05:58:32Z  
3         This is comment 4  Hrohaan Malhotra  2024-06-17T05:58:41Z  


In [30]:
df=[]

In [31]:
from zipfile import ZipFile
from lxml import etree
import pandas as pd
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_comments_and_highlighted_text(docx_file):
    data = []

    # Extract comments using lxml
    comments = {}
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/comments.xml' in zipf.namelist():
            with zipf.open('word/comments.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                for comment in root.findall('.//w:comment', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                    comment_id = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', '')
                    author = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}author', '')
                    date = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}date', '')

                    comment_text = ""
                    for text_element in comment.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        if text_element.text:
                            comment_text += text_element.text

                    comments[comment_id] = {
                        'Author': author,
                        'Date': date,
                        'Text': comment_text
                    }
        else:
            print("No comments.xml found in the document.")

    # Extract highlighted text and match it to comments
    highlighted_texts = []
    doc = Document(docx_file)
    comment_index = 0
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                highlighted_texts.append({
                    'Highlighted Text': run.text,
                    'Comment ID': None
                })

    # Read main document XML to map comments to text
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/document.xml' in zipf.namelist():
            with zipf.open('word/document.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                current_comment_id = None
                for elem in root.iter():
                    if elem.tag.endswith('commentRangeStart'):
                        current_comment_id = elem.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', None)
                    elif elem.tag.endswith('commentRangeEnd'):
                        current_comment_id = None
                    elif elem.tag.endswith('r') and current_comment_id:
                        for t in elem.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                            if t.text:
                                highlighted_texts.append({
                                    'Highlighted Text': t.text,
                                    'Comment ID': current_comment_id
                                })

    # Pair highlighted text with comments
    for highlighted in highlighted_texts:
        comment_id = highlighted['Comment ID']
        if comment_id and comment_id in comments:
            data.append({
                'Highlighted Text': highlighted['Highlighted Text'],
                'Comment Text': comments[comment_id]['Text'],
                'Comment Author': comments[comment_id]['Author'],
                'Comment Date': comments[comment_id]['Date']
            })
        else:
            data.append({
                'Highlighted Text': highlighted['Highlighted Text'],
                'Comment Text': 'N/A',
                'Comment Author': 'N/A',
                'Comment Date': 'N/A'
            })

    # Create a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Example usage
if __name__ == "__main__":
    docx_file = 'Document2.docx'
    df = extract_comments_and_highlighted_text(docx_file)
    
    print("HIGHLIGHTED TEXT AND COMMENTS PAIRS:")
    print(df)


HIGHLIGHTED TEXT AND COMMENTS PAIRS:
                                    Highlighted Text  \
0  That’s why this moment on the internet feels e...   
1  It’s easy to name a root cause — ads reward an...   
2  Now we have decades of proof that attention-gr...   
3  Everything Medium does is paid for by our memb...   
4  That’s why this moment on the internet feels e...   
5  It’s easy to name a root cause — ads reward an...   
6  Now we have decades of proof that attention-gr...   
7  Everything Medium does is paid for by our memb...   

               Comment Text    Comment Author          Comment Date  
0                       N/A               N/A                   N/A  
1                       N/A               N/A                   N/A  
2                       N/A               N/A                   N/A  
3                       N/A               N/A                   N/A  
4  This is comment number 1  Hrohaan Malhotra  2024-06-17T05:58:09Z  
5  This is Comment number 2  Hrohaan M

In [34]:
from zipfile import ZipFile
from lxml import etree
import pandas as pd
from docx import Document
from docx.enum.text import WD_COLOR_INDEX

def extract_comments_and_highlighted_text(docx_file):
    data = []
    comments = {}
    highlighted_texts = []

    # Extract comments using lxml
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/comments.xml' in zipf.namelist():
            with zipf.open('word/comments.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                for comment in root.findall('.//w:comment', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                    comment_id = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', '')
                    author = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}author', '')
                    date = comment.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}date', '')

                    comment_text = ""
                    for text_element in comment.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                        if text_element.text:
                            comment_text += text_element.text

                    comments[comment_id] = {
                        'Author': author,
                        'Date': date,
                        'Text': comment_text
                    }
        else:
            print("No comments.xml found in the document.")

    # Extract highlighted text and match it to comments
    doc = Document(docx_file)
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if run.font.highlight_color == WD_COLOR_INDEX.YELLOW:  # Adjust as needed
                highlighted_texts.append({
                    'Highlighted Text': run.text,
                    'Comment ID': None
                })

    # Read main document XML to map comments to text
    with ZipFile(docx_file, 'r') as zipf:
        if 'word/document.xml' in zipf.namelist():
            with zipf.open('word/document.xml') as f:
                tree = etree.parse(f)
                root = tree.getroot()

                current_comment_id = None
                for elem in root.iter():
                    if elem.tag.endswith('commentRangeStart'):
                        current_comment_id = elem.attrib.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}id', None)
                    elif elem.tag.endswith('commentRangeEnd'):
                        current_comment_id = None
                    elif elem.tag.endswith('r') and current_comment_id:
                        for t in elem.findall('.//w:t', namespaces={'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}):
                            if t.text:
                                highlighted_texts.append({
                                    'Highlighted Text': t.text,
                                    'Comment ID': current_comment_id
                                })

    # Create a dictionary to avoid duplicates
    highlighted_texts_dict = {text['Highlighted Text']: text['Comment ID'] for text in highlighted_texts}

    # Pair highlighted text with comments
    for highlighted_text, comment_id in highlighted_texts_dict.items():
        if comment_id and comment_id in comments:
            data.append({
                'Highlighted Text': highlighted_text,
                'Comment Text': comments[comment_id]['Text'],
                'Comment Author': comments[comment_id]['Author'],
                'Comment Date': comments[comment_id]['Date']
            })
        else:
            data.append({
                'Highlighted Text': highlighted_text,
                'Comment Text': 'N/A',
                'Comment Author': 'N/A',
                'Comment Date': 'N/A'
            })

    # Create a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Example usage
if __name__ == "__main__":
    docx_file = 'Document2.docx'
    df = extract_comments_and_highlighted_text(docx_file)
    
    print("HIGHLIGHTED TEXT AND COMMENTS PAIRS:")
    print(df)


HIGHLIGHTED TEXT AND COMMENTS PAIRS:
                                    Highlighted Text  \
0  That’s why this moment on the internet feels e...   
1  It’s easy to name a root cause — ads reward an...   
2  Now we have decades of proof that attention-gr...   
3  Everything Medium does is paid for by our memb...   

               Comment Text    Comment Author          Comment Date  
0  This is comment number 1  Hrohaan Malhotra  2024-06-17T05:58:09Z  
1  This is Comment number 2  Hrohaan Malhotra  2024-06-17T05:58:20Z  
2         This is comment 4  Hrohaan Malhotra  2024-06-17T05:58:41Z  
3  This is comment number 3  Hrohaan Malhotra  2024-06-17T05:58:32Z  
